Preface: I used Python 3 (v 5.5.0) to write this project. Other used software are annotated below in **boldface**.

# RNAfold: Secondary Structure of Riboswitches

In this lab, I used _subprocess_ and **ViennaRNA RNAfold** to model the secondary structures of various riboswitches from Robert Penchovsky and Ronald Breaker's 2005 Nature Biotechnology article (["Computational design and experimental validation of oligonucleotide-sensing allosteric ribozymes"](https://www.nature.com/articles/nbt1155)). Figure references to those in the article.

Effector DNA binds to "OBS" (oligonucleotide binding sites). There are various DNA substrates used in the article that bind each pertinent OBS.

## Questions Regarding the Function of Riboswitches in Penchovsky et al.

The riboswitches in Penchovsky et al. have hammerhead domains which allow for catalytic self-cleavage when certain conditions are fulfilled. In this case, conditional binding of effector DNA to OBS sequences and binding of "red" effector regions to each other leads to self-cleavage.

_Will the AND-1 riboswitch cleave itself when both of its OBS are bound?_
AND-1 will only self-cleave when both OBS are bound.

_Will the OR-1 riboswitch cleave itself when neither of its OBS are bound?_
OR-1 will self-cleave when either OBS-1 or OBS-2 is bound.

_What behavior do we expect from the YES-1 riboswitch?_
YES-1 will self-cleave when the effector DNA strand binds the OBS-1 "faithfully." This leads to basepairing of the "red" effector regions to each other and subsequent self-cleavage.

## 1. Table of Riboswitches in Penchovsky et al.

In [1]:
# I created the database riboswitch.db to store all the riboswitch data from Breaker et al. Data 5' to 3', with labels corresponding to different OBS and "red" regions.

import sqlite3
conn = sqlite3.connect('riboswitch.db')
c = conn.cursor()

c.execute("""CREATE TABLE data (name TEXT PRIMARY KEY ASC, 
                                start_OBS1 TEXT,
                                end_OBS1 TEXT,
                                start_OBS2 TEXT, 
                                end_OBS2 TEXT, 
                                start_Red1 TEXT, 
                                end_Red1 TEXT, 
                                start_Red2 TEXT, 
                                end_Red2 TEXT,
                                strand VARCHAR(1));""")

# Column Labels

c.execute("""INSERT INTO data (name, start_OBS1, end_OBS1, start_OBS2, end_OBS2, start_Red1, end_Red1, start_Red2, end_Red2, strand)
                        VALUES ('Name','OBS1 Start','OBS1 End','OBS2 Start','OBS2 End','Red1 Start','Red1 End','Red2 Start','Red2 End','-');""")

# Figure 2a YES-1

c.execute("""INSERT INTO data (name, start_OBS1, end_OBS1, start_OBS2, end_OBS2, start_Red1, end_Red1, start_Red2, end_Red2, strand)
                        VALUES ('YES1','26','47','N/A','N/A','16','21','49','54','-');""")

# Figure 4a NOT-1

c.execute("""INSERT INTO data (name, start_OBS1, end_OBS1, start_OBS2, end_OBS2, start_Red1, end_Red1, start_Red2, end_Red2, strand)
                        VALUES ('NOT1','44','66','N/A','N/A','40','43','74','77','-');""")

# Figure 5a AND-1

c.execute("""INSERT INTO data (name, start_OBS1, end_OBS1, start_OBS2, end_OBS2, start_Red1, end_Red1, start_Red2, end_Red2, strand)
                        VALUES ('AND1','30','45','49','64','16','23','70','77','-');""")

# Figure 6a OR-1

c.execute("""INSERT INTO data (name, start_OBS1, end_OBS1, start_OBS2, end_OBS2, start_Red1, end_Red1, start_Red2, end_Red2, strand)
                        VALUES ('OR1','27','46','47','66','16','26','67','77','-');""")

conn.commit()

# Output table below

c.execute("SELECT * FROM data;")
list=(c.fetchall())
for i in list:
    print(i)

('Name', 'OBS1 Start', 'OBS1 End', 'OBS2 Start', 'OBS2 End', 'Red1 Start', 'Red1 End', 'Red2 Start', 'Red2 End', '-')
('YES1', '26', '47', 'N/A', 'N/A', '16', '21', '49', '54', '-')
('NOT1', '44', '66', 'N/A', 'N/A', '40', '43', '74', '77', '-')
('AND1', '30', '45', '49', '64', '16', '23', '70', '77', '-')
('OR1', '27', '46', '47', '66', '16', '26', '67', '77', '-')


## 2. RNAfold: Without OBS Binding

In [2]:
# This cell runs RNAfold in iPython using the subprocess script. No OBS binding is considered here.

import subprocess

# Input sequences to be modelled by RNAfold

seqs=\
""">YES1
gggcgacccugaugagcuugaguuuagcucgucacuguccagguucaaucaggcgaaacggugaaagccguagguugccc
>NOT1
ggcagguacauacagcugaugagucccaaauaggacgaaacgcgacacacaccacuaaaccgugcaguguuuugcguccuguauuccacugc
>AND1
gggcgacccugaugagcuugguuuaguauuuacagcuccauacaugagguguuaucccuaugcaaguucgaucaggcgaaacggugaaagccguagguugcccagagacaau
>OR1
gggcgacccugaugagcuugguugaguauuuacagcuccauacaugagguguucucccuacgcaaguucgaucaggcgaaacggugaaagccguagguugccc
"""

p=subprocess.run(['RNAfold'],
                input=bytes(seqs,'ascii'),
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

# Output folding coordinates and modelled structure 

print(p.stderr.decode())

print(p.stdout.decode())


>YES1
gggcgacccugaugagcuugaguuuagcucgucacuguccagguucaaucaggcgaaacggugaaagccguagguugccc
((((((((.(((((((((.......))))))))).((.((.(((...))).))))..(((((....))))).)))))))) (-33.00)
>NOT1
ggcagguacauacagcugaugagucccaaauaggacgaaacgcgacacacaccacuaaaccgugcaguguuuugcguccuguauuccacugc
.((((....((((((.......((((......))))...((((((.((((..(((......)))..)))).)))))).))))))....)))) (-28.10)
>AND1
gggcgacccugaugagcuugguuuaguauuuacagcuccauacaugagguguuaucccuaugcaaguucgaucaggcgaaacggugaaagccguagguugcccagagacaau
((((((((((((((((((((...(((.....(((.(((.......))).))).....)))..))))))).))))).....(((((....))))).))))))))......... (-42.10)
>OR1
gggcgacccugaugagcuugguugaguauuuacagcuccauacaugagguguucucccuacgcaaguucgaucaggcgaaacggugaaagccguagguugccc
((((((((((((((((((((((.(((.....(((.(((.......))).))).)))...)).))))))).))))).....(((((....))))).)))))))) (-40.00)



### Parameter Differences between RNAfold and Published Models

The default temperature used in **RNAfold** and the published models is 37°C, though the authors did use **RNAheat** to model riboswitch secondary structures from 20-40°C. 

The default algorithm in **RNAfold** is minimizing the minimum free energy of the structure, while the published models uses McCaskill's equilibrium partition model.

Slight differences in parameterization from **RNAfold** default settings may have contributed to structural differences between the two models. The authors state that they "essentially [used] the RNAfold source code."

### Unbound YES-1 Gate

<img src="YES1_ss.jpg" width="30%">


There are some major differences between this structure and the YES-1 gate in _Figure 2a_. In particular, the step-loop region between 10 and 50 nucleotides are deviant. The article shows an interior loop (22-25, 37-39 nt) and small hairpin (27-35 nt), while the **RNAfold** model shows a large hairpin (18-26 nt) and basepairing until the nucleotide 34. The large interior loop is larger in the **RNAfold** model than the actual published model. There are also additional loops in the **RNAfold** model between nucleotides 40-52. These deviations lead to the existence of an extra stem in the **RNAfold** model. It should be noted, though, that the hairpin between nucleotides 62-67 is identical in both models. 

The "red" regions are not bound to each other, and there is no output of TRUE signal.

Deviations may be due to different kinetic stability of the two structures based on different parameters used by **RNAfold** and Penchovsky et al.

### Unbound NOT-1 Gate

<img src="NOT1_ss.jpg" width="30%">

This structure is identical to the NOT-1 gate in _Figure 4a_. The "red" regions are not bound to each other, and there is no output of TRUE signal.

### Unbound AND-1 Gate (F/F)

<img src="AND1_ss.jpg" width="30%">

This structure is identical to the AND-1 gate in _Figure 5a_. The "red" regions are not bound to each other, and there is no output of TRUE signal.

### Unbound OR-1 Gate (F/F)

<img src="OR1_ss.jpg" width="30%">

This structure is identical to the OR-1 gate in _Figure 6a_. The "red" regions are not bound to each other, and there is no output of TRUE signal.

## 3. RNAfold: OBS Binding of YES-1 and NOT-1

### Bound YES-1 Gate: 

Using **MacOS Terminal**, I ran the code "RNAfold -C YES-1_constraint.txt" with the sequence below to give the structure.

GGGCGACCCUGAUGAGCUUGAGUUUAGCUCGUCACUGUCCAGGUUCAAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
.........................xxxxxxxxxxxxxxxxxxxxxx.................................

Where "." represents unconstrained bases and "x" unpaired bases. "x" bases represent the nucleotides that bind to effector DNA. 

<img src="YES1_constraint_ss.jpg" width="30%">

This structure is identical to the self-cleaving form of YES-1 gate in _Figure 2a_. The "red" regions are  bound to each other, and there is thus output of TRUE signal.

### Bound NOT-1 Gate: 

"RNAfold -C NOT-1_constraint.txt"
GGCAGGUACAUACAGCUGAUGAGUCCCAAAUAGGGACGAACGCGACACACACCACUAAACCGUGCAGUGUUUUGCGUCCUGUAUUCCACUGC
............................................xxxxxxxxxxxxxxxxxxxxxx..........................

<img src="NOT1_constraint_ss.jpg" width="30%">

This structure is identical to the self-cleaving form of NOT-1 gate in _Figure 4a_. The "red" regions are not bound to each other, and there is no output of TRUE signal.

## 4. RNAfold: OBS Binding of AND-1 and OR-1

Similar to **Section 3**, I used **MacOS Terminal** to determine the structures of AND-1 and OR-1 gates when OBS-1 and/or OBS-2 are bound. Because there are three other possible permutations (T/F, F/T, T/T; F/F is modelled in **Section 2**), three separate constraints .txt files were made and fed to **RNAfold** for each gate.

T refers to OBS binding, while F to lack of OBS binding. _/_ refers to binding to OBS-1/OBS-2.

### Bound AND-1 Gates

"RNAfold -C AND-1_constraints1.txt": **T/F, OBS-1 binding only**
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACUAGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
....................xxxxxxxxxxxxxxxxxxxxxxxxx...................................................................

"RNAfold -C AND-1_constraints2.txt": **F/T, OBS-2 binding only**
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACUAGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
.................................................xxxxxxxxxxxxxxx................................................

"RNAfold -C AND-1_constraints3.txt": **T/T, OBS-1 and OBS-2 binding**
GGGCGACCCUGAUGAGCUUGGUUUAGUAUUUACAGCUCCAUACUAGAGGUGUUAUCCCUAUGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCCAGAGACAAU
....................xxxxxxxxxxxxxxxxxxxxxxxxx....xxxxxxxxxxxxxxx................................................

**T/F** <img src="AND1_constraints1.jpg" width="30%">

**F/T** <img src="AND1_constraints2.jpg" width="30%">

**T/T** <img src="AND1_constraints3.jpg" width="30%">

None of the **RNAfold** AND-1 gates are identical to those in _Figure 5a_. However, the third constraint (T/T) shows that binding to both OBS-1 and OBS-2 leads to the output of TRUE signal (that is, the "red" regions bind). Likewise, the "red" regions do not bind for the T/F or F/T constraints. 

**_This confirms that the article's AND-1 gate works properly._**

### Bound OR-1 Gates

"RNAfold -C OR1_constraints1.txt": **T/F, OBS-1 binding only**
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
...........................xxxxxxxxxxxxxxxxxxx.........................................................

"RNAfold -C OR1_constraints2.txt": **F/T, OBS-2 binding only**
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
..............................................xxxxxxxxxxxxxxxxxxx......................................

"RNAfold -C OR1_constraints3.txt": **T/T, OBS-1 and OBS-2 binding**
GGGCGACCCUGAUGAGCUUGGUUGAGUAUUUACAGCUCCAUACAUGAGGUGUUCUCCCUACGCAAGUUCGAUCAGGCGAAACGGUGAAAGCCGUAGGUUGCCC
...........................xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx......................................

**T/F** <img src="OR1_constraints1.jpg" width="30%">

**F/T** <img src="OR1_constraints2.jpg" width="30%">

**T/T** <img src="OR1_constraints3.jpg" width="30%">

None of the **RNAfold** OR-1 gates are identical to those in _Figure 6a_. Binding to OBS-2 is identical in both sets of structures, but OBS-1 binding is not. Thus, two of the three constraints (F/T, T/T)  result in "red" region binding and TRUE signal output upon effector DNA binding to OBS-2.

However, the OBS-1 binding only constraint (T/F) does not lead to "red" region binding in our **RNAfold** model. 

**_This raises questions about the efficacy of the article's OR-1 gate._**